In [6]:
import os
import random
import math

import numpy as np
import haven.db as db 
import plotly.express as px

from mirrorverse.utils import read_data_w_cache

os.environ['HAVEN_DATABASE'] = 'haven'
os.environ['AWS_PROFILE'] = 'admin'

In [14]:
TEST = True

table = 'movement_model_raw_features_m2_a2' if not TEST else 'movement_model_raw_features_m2_a2_test'
data = read_data_w_cache(
    f'select * from {table}'
)
data = data[data['_selected'].isin([True, False])]
data.head()

,_decision,_choice,_selected,_individual,mixed_layer_thickness,net_primary_production,water_heading,movement_heading,distance,origin_h3_index,next_h3_index,h3_index,time,_train,tag_key
0,4,7,False,229209,10.533973,8.875200,2.997269,-2.200149,67.699165,8412b29ffffffff,8412b29ffffffff,8412b25ffffffff,2022-05-30,False,229209
1,21,13,False,229209,10.528886,33.575108,2.327530,-0.333718,42.447239,8412b3dffffffff,8412b3dffffffff,8412b31ffffffff,2022-06-16,False,229209
2,12,14,False,229209,10.528886,7.610131,1.937519,2.827055,41.842129,8412b25ffffffff,8412b25ffffffff,841d2cbffffffff,2022-06-07,False,229209
3,75,1,False,210765,10.008276,21.698296,2.894555,0.000000,0.000000,840c681ffffffff,841d34bffffffff,840c681ffffffff,2021-05-18,False,210765
4,90,11,False,205413,32.006288,0.864287,-2.282105,-0.796432,38.843296,840c43dffffffff,840c423ffffffff,840c431ffffffff,2021-01-03,False,205413


In [15]:
df = data.merge(
    data[['_individual']].drop_duplicates().reset_index(drop=True).reset_index(),
    on='_individual'
)
del df['_individual']
df = df.rename({'index': '_individual'}, axis=1)
df.head()

,_decision,_choice,_selected,mixed_layer_thickness,net_primary_production,water_heading,movement_heading,distance,origin_h3_index,next_h3_index,h3_index,time,_train,tag_key,_individual
0,4,7,False,10.533973,8.875200,2.997269,-2.200149,67.699165,8412b29ffffffff,8412b29ffffffff,8412b25ffffffff,2022-05-30,False,229209,0
1,21,13,False,10.528886,33.575108,2.327530,-0.333718,42.447239,8412b3dffffffff,8412b3dffffffff,8412b31ffffffff,2022-06-16,False,229209,0
2,12,14,False,10.528886,7.610131,1.937519,2.827055,41.842129,8412b25ffffffff,8412b25ffffffff,841d2cbffffffff,2022-06-07,False,229209,0
3,75,1,False,10.008276,21.698296,2.894555,0.000000,0.000000,840c681ffffffff,841d34bffffffff,840c681ffffffff,2021-05-18,False,210765,1
4,90,11,False,32.006288,0.864287,-2.282105,-0.796432,38.843296,840c43dffffffff,840c423ffffffff,840c431ffffffff,2021-01-03,False,205413,2


In [16]:
df.groupby('_train').size()

_train
False    25112
dtype: int64

In [17]:
df['normed_distance'] = df['distance'] / 100
px.histogram(df['normed_distance'])

In [18]:
df['normed_log_mlt'] = np.log(df['mixed_layer_thickness'] + 0.001)
print(df['normed_log_mlt'].mean())
df['normed_log_mlt'] = df['normed_log_mlt'] - 3.068049438368808
px.histogram(df['normed_log_mlt'])

3.2224708769130967


In [19]:
df['normed_log_npp'] = np.log(df['net_primary_production'] + 0.001)
print(df['normed_log_npp'].mean())
df['normed_log_npp'] = df['normed_log_npp'] - 1.9856236
px.histogram(df['normed_log_npp'])

1.8876421


In [20]:
table = 'movement_model_features_m2_a2' if not TEST else 'movement_model_features_m2_a2_test'

db.write_data(
    df, table, ['tag_key']
)